<img style="float: right;" src="images/hyperstream.svg">

# HyperStream Tutorial 5: Workflows

Workflows define a graph of streams. Usually, the first stream will be a special "raw" stream that pulls in data from a custom data source. Workflows can have multiple time ranges, which will cause the streams to be computed on all of the ranges given.

## Introduction

In this tutorial, we will be ussing a time-series dataset about the temperature in different countries and cities. The dataset is availabel at [The Census at School New Zeland][1]. The necessary files for this tutorial are already included in the folder **data/TimeSeriesDatasets_130207**.

In particular, there are four files with the minimum and maximum temperatures in different cities of Asia, Australia, NZ and USA from 2000 to 2012. And the rainfall levels of New Zeland. 

![workflows](images/workflow_world_temp.svg)

[1]: http://new.censusatschool.org.nz/resource/time-series-data-sets-2013/

In [1]:
try:
    %load_ext watermark
    watermark = True
except ImportError:
    watermark = False
    pass

import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream
from hyperstream import TimeInterval
from hyperstream.utils import UTC
import hyperstream

from datetime import datetime
from utils import plot_high_chart
from utils import plot_multiple_stock
from dateutil.parser import parse

if watermark:
    %watermark -v -m -p hyperstream -g

hs = HyperStream(loglevel=30)
print(hs)
print([p.channel_id_prefix for p in hs.config.plugins])

CPython 2.7.6
IPython 5.4.1

hyperstream 0.3.7

compiler   : GCC 4.8.4
system     : Linux
release    : 3.19.0-80-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : 3c1d59ac078ff69e373726be200d8ac025627ef9
HyperStream version 0.3.7, connected to mongodb://localhost:27017/hyperstream, session id <no session>
[u'example', u'data_importers', u'data_generators']


## Reading the data

In the data folder there are four csv files with the names **TempAsia.csv, TempAustralia.csv, TempNZ.csv and TempUSA.csv**. The first column of each csv file contains a header with the names of the columns. The first one being the date and the following are the minimum and maximum temperature in different cities with the format **cityMin** and **cityMax**.

Here is an example of the first 5 rows of the **TempAsia.csv** file:

```
Date,TokyoMax,TokyoMin,BangkokMax,BangkokMin
2000M01,11.2,4.2,32.8,24
```

The format of the date has the form **YYYYMmm** where **YYYY** is the year and **mm** is the month. Because this format is not recognized by the default parser of the **csv_reader** tool, we will need to specify our own parser that first replaces the **M** by an hyphen **-** and then applies the **dateutils.parser**.

Then, we will use a tool to read each csv, and a Stream to store all the results of applying the tool. When we specify to the tool that there is a header row in the csv file, the value of each Stream instance will be a dictionary with the name of the column and its corresponding value. For example, a Stream instance with the 4 cities shown above will look like:

```
[2000-01-19 00:00:00+00:00]: {'BangkokMin': 24.0, 'BangkokMax': 32.8, 'TokyoMin': 4.2}
```

In [2]:
def dateparser(dt):
    return parse(dt.replace('M', '-')).replace(tzinfo=UTC)

ti_all = TimeInterval(datetime(1999, 1, 1).replace(tzinfo=UTC),
                      datetime(2013, 1, 1).replace(tzinfo=UTC))
ti_sample = TimeInterval(datetime(2007, 1, 1).replace(tzinfo=UTC),
                         datetime(2007, 3, 1).replace(tzinfo=UTC))

# M will be the Memory Channel
M = hs.channel_manager.memory

countries_list = ['Asia', 'Australia', 'NZ', 'USA']

## Create the plates and meta_data instances

In [3]:
countries_dict = {
    'Asia': ['BangkokMin', 'BangkokMax', 'HongKongMax', 'HongKongMin', 'KualaLumpurMax', 'KualaLumpurMin',
             'NewDelhiMax', 'NewDelhiMin', 'TokyoMax', 'TokyoMin'],
    'Australia': ['BrisbaneMax', 'BrisbaneMin', 'Canberramax', 'CanberraMin', 'GoldCoastMax', 'GoldCoastMin',
                  'MelbourneMin', 'Melbournemax',  'SydneyMax', 'SydneyMin'],
    'NZ': ['AucklandMax', 'AucklandMin', 'ChristchurchMax', 'ChristchurchMin', 'DunedinMax', 'DunedinMin',
           'HamiltonMax', 'HamiltonMin','WellingtonMax', 'WellingtonMin',
           'Auckland', 'Christchurch', 'Dunedin', 'Hamilton','Wellington'],
    'USA': ['ChicagoMin', 'ChicagoMax', 'HoustonMax', 'HoustonMin', 'LosAngelesMax', 'LosAngelesMin',
            'NYMax', 'NYMin', 'SeattleMax', 'SeattleMin']
}

# delete_plate requires the deletion to be first childs and then parents
for plate_id in ['C.C', 'C']:
    if plate_id in [plate[0] for plate in hs.plate_manager.plates.items()]:
        hs.plate_manager.delete_plate(plate_id=plate_id, delete_meta_data=True)

for country in countries_dict:
    id_country = 'country_' + country
    if not hs.plate_manager.meta_data_manager.contains(identifier=id_country):
        hs.plate_manager.meta_data_manager.insert(
            parent='root', data=country, tag='country', identifier=id_country)
    for city in countries_dict[country]:
        id_city = id_country + '.' + 'city_' + city
        if not hs.plate_manager.meta_data_manager.contains(identifier=id_city):
            hs.plate_manager.meta_data_manager.insert(
                parent=id_country, data=city, tag='city', identifier=id_city)
            
C = hs.plate_manager.create_plate(plate_id="C", description="Countries", values=[], complement=True,
                                  parent_plate=None, meta_data_id="country")
CC = hs.plate_manager.create_plate(plate_id="C.C", description="Cities", values=[], complement=True,
                                   parent_plate="C", meta_data_id="city")

In [4]:
print hs.plate_manager.meta_data_manager.global_plate_definitions


root[root:None]
╟── country[country_NZ:NZ]
║   ╟── city[country_NZ.city_AucklandMax:AucklandMax]
║   ╟── city[country_NZ.city_AucklandMin:AucklandMin]
║   ╟── city[country_NZ.city_ChristchurchMax:ChristchurchMax]
║   ╟── city[country_NZ.city_ChristchurchMin:ChristchurchMin]
║   ╟── city[country_NZ.city_DunedinMax:DunedinMax]
║   ╟── city[country_NZ.city_DunedinMin:DunedinMin]
║   ╟── city[country_NZ.city_HamiltonMax:HamiltonMax]
║   ╟── city[country_NZ.city_HamiltonMin:HamiltonMin]
║   ╟── city[country_NZ.city_WellingtonMax:WellingtonMax]
║   ╟── city[country_NZ.city_WellingtonMin:WellingtonMin]
║   ╟── city[country_NZ.city_Auckland:Auckland]
║   ╟── city[country_NZ.city_Christchurch:Christchurch]
║   ╟── city[country_NZ.city_Dunedin:Dunedin]
║   ╟── city[country_NZ.city_Hamilton:Hamilton]
║   ╙── city[country_NZ.city_Wellington:Wellington]
╟── country[country_Australia:Australia]
║   ╟── city[country_Australia.city_BrisbaneMax:BrisbaneMax]
║   ╟── city[country_Australia.city_Brisbane

## Create the workflow and execute it

In [5]:
from hyperstream import Workflow

# parameters for the csv_mutli_reader tool
csv_temp_params = dict(
    filename_template='data/TimeSeriesDatasets_130207/Temp{}.csv',
    datetime_parser=dateparser, skip_rows=0, header=True)

csv_rain_params = dict(
    filename_template='data/TimeSeriesDatasets_130207/{}Rainfall.csv',
    datetime_parser=dateparser, skip_rows=0, header=True)

def mean(x):
    x = [value for value in x if value is not None]
    return float(sum(x)) / max(len(x), 1)


with Workflow(workflow_id='tutorial_05',
              name='tutorial_05',
              owner='tutorials',
              description='Tutorial 5 workflow',
              online=False) as w:

    country_node_raw_temp = w.create_node(stream_name='raw_temp_data', channel=M, plates=[C])
    country_node_raw_rain = w.create_node(stream_name='raw_rain_data', channel=M, plates=[C])
    city_node_temp = w.create_node(stream_name='city_temp', channel=M, plates=[CC])
    city_node_rain = w.create_node(stream_name='city_rain', channel=M, plates=[CC])
    country_node_avg_temp = w.create_node(stream_name='country_avg_temp', channel=M, plates=[C])
    country_node_avg_rain = w.create_node(stream_name='country_avg_rain', channel=M, plates=[C])
    # FIXME Should I create a node outside the plates?
    world_node_avg_temp = w.create_node(stream_name='world_avg_temp',
                                        channel=M, plates=None)

    for c in C:
        country_node_raw_temp[c] = hs.plugins.data_importers.factors.csv_multi_reader(
                source=None, **csv_temp_params)
        country_node_raw_rain[c] = hs.plugins.data_importers.factors.csv_multi_reader(
                source=None, **csv_rain_params)
        for cc in CC[c]:
            city_node_temp[cc] = hs.factors.splitter_from_stream(
                                    source=country_node_raw_temp[c],
                                    splitting_node=country_node_raw_temp[c],
                                    use_mapping_keys_only=True)
            city_node_rain[cc] = hs.factors.splitter_from_stream(
                                    source=country_node_raw_rain[c],
                                    splitting_node=country_node_raw_rain[c],
                                    use_mapping_keys_only=True)
        country_node_avg_temp[c] = hs.factors.aggregate(
                                    sources=[city_node_temp],
                                    alignment_node=None,
                                    aggregation_meta_data='city', func=mean)
        country_node_avg_rain[c] = hs.factors.aggregate(
                                    sources=[city_node_rain],
                                    alignment_node=None,
                                    aggregation_meta_data='city', func=mean)
    # FIXME Should I create a node for a stream outside the plates?
    world_node_avg_temp = hs.factors.aggregate(sources=[country_node_avg_temp],
                                               alignment_node=None,
                                               aggregation_meta_data='country',
                                               func=mean)
    w.execute(ti_all)

## See the country temperature and rain

In [6]:
# Print the results
for stream_id, stream in country_node_avg_temp.streams.iteritems():
    print(stream_id)
    print(stream.window().first())

(('country', 'NZ'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 16, 0, 0, tzinfo=<UTC>), value=16.729999999999997)
(('country', 'USA'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 16, 0, 0, tzinfo=<UTC>), value=6.0600000000000005)
(('country', 'Australia'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 16, 0, 0, tzinfo=<UTC>), value=21.240000000000002)
(('country', 'Asia'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 16, 0, 0, tzinfo=<UTC>), value=18.990000000000002)


In [7]:
for stream_id, stream in country_node_avg_rain.streams.iteritems():
    print(stream_id)
    print(stream.window().first())

(('country', 'NZ'),)
StreamInstance(timestamp=datetime.datetime(2000, 1, 16, 0, 0, tzinfo=<UTC>), value=105.08)
(('country', 'USA'),)
None
(('country', 'Australia'),)
None
(('country', 'Asia'),)
None


In [8]:
def get_x_y_names_from_streams(streams, tag=None):
    names = []
    y = []
    x = []
    for stream_id, stream in streams.iteritems():
        if len(stream.window().items()) == 0:
            continue
        meta_data = dict(stream_id.meta_data)
        names.append(meta_data[tag])
        y.append([instance.value for instance in stream.window().items()])
        x.append([str(instance.timestamp) for instance in stream.window().items()])
    return y, x, names

data, time, names = get_x_y_names_from_streams(M.find_streams(country='NZ', name='city_temp'), 'city')

plot_multiple_stock(data, time=time, names=names,
                    title='Temperatures in New Zealand', ylabel='ºC')

In [9]:
data, time, names = get_x_y_names_from_streams(M.find_streams(country='Australia', name='city_temp'), 'city')

plot_multiple_stock(data, time=time, names=names,
                    title='Temperatures in Australia', ylabel='ºC')

In [10]:
data, time, names = get_x_y_names_from_streams(M.find_streams(country='NZ', name='city_rain'), 'city')

plot_multiple_stock(data, time=time, names=names,
                    title='Rain in New Zealand', ylabel='some precipitation unit')

In [11]:
data, time, names = get_x_y_names_from_streams(M.find_streams(name='country_avg_temp'), 'country')

plot_multiple_stock(data, time=time, names=names,
                    title='Temperatures in countries', ylabel='ºC')

In [12]:
data, time, names = get_x_y_names_from_streams(M.find_streams(name='country_avg_rain'), 'country')

plot_multiple_stock(data, time=time, names=names,
                    title='Average rain in countries', ylabel='some precipitation unit')

In [13]:
for stream_id, stream in M.find_streams(name='world_avg_temp').iteritems():
    print stream_id
    print [instance.value for instance in stream.window(ti_sample).items()]

2017-08-16 18:51:15,619 [WARNI]  Stream world_avg_temp not available for time interval (2007-01-01 00:00:00+00:00, 2007-03-01 00:00:00+00:00]. Perhaps upstream calculations haven't been performed


world_avg_temp
[]


In [14]:
from pprint import pprint
pprint(w.to_dict(tool_long_names=False))

{'factors': [{'id': 'csv_multi_reader',
              'sink': 'raw_temp_data',
              'sources': []},
             {'id': 'csv_multi_reader',
              'sink': 'raw_rain_data',
              'sources': []},
             {'id': 'splitter_from_stream',
              'sink': 'city_temp',
              'sources': ['raw_temp_data']},
             {'id': 'splitter_from_stream',
              'sink': 'city_rain',
              'sources': ['raw_rain_data']},
             {'id': 'aggregate',
              'sink': 'country_avg_temp',
              'sources': ['city_temp']},
             {'id': 'aggregate',
              'sink': 'country_avg_rain',
              'sources': ['city_rain']}],
 'nodes': [{'id': 'raw_rain_data'},
           {'id': 'country_avg_temp'},
           {'id': 'country_avg_rain'},
           {'id': 'city_rain'},
           {'id': 'city_temp'},
           {'id': 'world_avg_temp'},
           {'id': 'raw_temp_data'}],
 'plates': {u'C': [{'id': 'raw_rain_data', 'type'

In [15]:
print(w.to_json(w.factorgraph_viz, tool_long_names=False, indent=4))

{
    "nodes": [
        {
            "type": "rv",
            "id": "raw_rain_data"
        },
        {
            "type": "rv",
            "id": "country_avg_temp"
        },
        {
            "type": "rv",
            "id": "country_avg_rain"
        },
        {
            "type": "rv",
            "id": "city_rain"
        },
        {
            "type": "rv",
            "id": "city_temp"
        },
        {
            "type": "rv",
            "id": "world_avg_temp"
        },
        {
            "type": "rv",
            "id": "raw_temp_data"
        },
        {
            "type": "fac",
            "id": "csv_multi_reader"
        },
        {
            "type": "fac",
            "id": "csv_multi_reader"
        },
        {
            "type": "fac",
            "id": "splitter_from_stream"
        },
        {
            "type": "fac",
            "id": "splitter_from_stream"
        },
        {
            "type": "fac",
            "id": "aggregate"
  